##Importando as bibliotecas

In [ ]:
import cv2
import numpy as np
from google.colab import drive
from google.colab.patches import cv2_imshow
import zipfile
import shutil
import os
import random
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Instalando a biblioteca do ultralytics para utilizar o yolo

In [ ]:
!pip install ultralytics
import ultralytics

Copiando as pastas que serão usadas para o ambiente de execução.

In [ ]:
shutil.copytree('/content/drive/MyDrive/yolo_data', '/content/yolo_data')
shutil.copytree('/content/drive/MyDrive/folds_A2A3', '/content/folds_A2A3')

'/content/folds_A2A3'

#Funções que vão auxiliar, essa primeira é para resetar a pasta, que no caso apaga todos arquivos para entrar outro k-fold.

In [ ]:
def resetar_pasta(pasta):

  if os.path.exists(pasta):
      for arquivo in os.listdir(pasta):
          caminho_arquivo = os.path.join(pasta, arquivo)
          if os.path.isfile(caminho_arquivo):
              os.remove(caminho_arquivo)

#Função para pegar os dados de treino e dividir entre treino e validação.

In [ ]:
def dividir_train_val(pasta_imagens, pasta_textos, train_images, train_labels, val_images, val_labels, proporcao_val):

    arquivos_imagens = [arquivo for arquivo in os.listdir(pasta_imagens) if arquivo.endswith(".jpg")]

    arquivos_textos = [arquivo[:-4] + ".txt" for arquivo in arquivos_imagens if os.path.exists(os.path.join(pasta_textos, arquivo[:-4] + ".txt"))]

    quantidade_val = int(len(arquivos_imagens) * proporcao_val)

    arquivos_val = random.sample(arquivos_imagens, quantidade_val)

    for arquivo_imagem in arquivos_imagens:
        caminho_imagem = os.path.join(pasta_imagens, arquivo_imagem)
        caminho_texto = os.path.join(pasta_textos, arquivo_imagem[:-4] + ".txt")
        if arquivo_imagem in arquivos_val:
            shutil.move(caminho_imagem, os.path.join(val_images, arquivo_imagem))
            shutil.move(caminho_texto, os.path.join(val_labels, arquivo_imagem[:-4] + ".txt"))
        else:
            shutil.move(caminho_imagem, os.path.join(train_images, arquivo_imagem))
            shutil.move(caminho_texto, os.path.join(train_labels, arquivo_imagem[:-4] + ".txt"))

#Função para contar linhas de um arquivo, criado para contar a quantidade de carros encontrados e quanto deveria achar.

In [ ]:
def contar_linhas_arquivo(nome_arquivo):
    with open(nome_arquivo, 'r') as arquivo:
        linhas = arquivo.readlines()
    return len(linhas)

#Função para treinar a IA, dividindo as pastas tudo certinho.


In [ ]:
def treinar(test_fold):
    folds_directory = "/content/folds_A2A3"
    resetar_pasta("/content/yolo_data/images/train")
    resetar_pasta("/content/yolo_data/images/test")
    resetar_pasta("/content/yolo_data/images/val")
    resetar_pasta("/content/yolo_data/labels/train")
    resetar_pasta("/content/yolo_data/labels/test")
    resetar_pasta("/content/yolo_data/labels/val")
    if os.path.isfile("/content/yolo_data/labels/train.cache"):
        os.remove("/content/yolo_data/labels/train.cache")
        os.remove("/content/yolo_data/labels/val.cache")

    data_images = "/content/yolo_data/images"
    data_labels = "/content/yolo_data/labels"
    test_images = "/content/yolo_data/images/test"
    test_labels = "/content/yolo_data/labels/test"

    test_fold = 4

    # Itera sobre os folds selecionados
    for fold_number in range(1, 6):

        fold_images = os.path.join(folds_directory, f"fold{fold_number}_images")
        fold_labels = os.path.join(folds_directory, f"fold{fold_number}_labels")

        images_list = os.listdir(fold_images)
        labels_list = os.listdir(fold_labels)

        if fold_number == test_fold:
          for file_name in images_list:

            source_file = os.path.join(fold_images, file_name)

            destination_file = os.path.join(test_images, file_name)

            shutil.copy(source_file, destination_file)

          for file_name in labels_list:

            source_file = os.path.join(fold_labels, file_name)

            destination_file = os.path.join(test_labels, file_name)

            shutil.copy(source_file, destination_file)

        else:
          for file_name in images_list:

            source_file = os.path.join(fold_images, file_name)

            destination_file = os.path.join(data_images, file_name)

            shutil.copy(source_file, destination_file)

          for file_name in labels_list:

            source_file = os.path.join(fold_labels, file_name)

            destination_file = os.path.join(data_labels, file_name)

            shutil.copy(source_file, destination_file)

          images = "/content/yolo_data/images"
          label = "/content/yolo_data/labels"

          train_img = "/content/yolo_data/images/train"
          train_label = "/content/yolo_data/labels/train"

          val_img = "/content/yolo_data/images/val"
          val_label = "/content/yolo_data/labels/val"

          dividir_train_val(images, label, train_img, train_label, val_img, val_label, 0.2)

    !yolo task=detect mode=train model=yolov8n.pt data=/content/drive/MyDrive/custom.yaml epochs=20 imgsz=640 project=/content/drive/MyDrive/Treinos

#Função para uma melhor visualização dos resultados.


In [ ]:
def ver_teste(pasta_teste, pasta_predict, conf):

    porcentagens_acerto = []
    contador = 0
    media_erros = 0

    for filename in os.listdir(pasta_teste):
        contador += 1
        predict = 0
        verdade = 0
        caminho_verdade = os.path.join(pasta_teste, filename)
        caminho_predict = os.path.join(pasta_predict, filename)

        if filename.endswith('.txt') and os.path.isfile(caminho_verdade):
            verdade = contar_linhas_arquivo(caminho_verdade)

        if filename.endswith('.txt') and os.path.isfile(caminho_predict):
            predict = contar_linhas_arquivo(caminho_predict)

        dif = verdade - predict
        nome_arquivo = os.path.splitext(os.path.basename(caminho_verdade))[0]

        #print("Arquivo:", nome_arquivo)
        #print("Diferença = ", abs(dif))

        media_erros += dif

        #print("Porcentagem de Acerto = ", porcentagem_acerto,"%")
        #print("--------------------------------------------")

    media_erros = media_erros / contador
    media_erros = round(media_erros, 2)
    print(f"Média de erros (CONF == {conf}) =", media_erros)

#Função para treinar, fazendo o cross-validation.

In [ ]:
for i in range(1, 6):
  treinar(i)

Ultralytics YOLOv8.0.124 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/custom.yaml, epochs=20, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Treinos, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, o